In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import pickle
import os
import osmnx as ox
from pyrosm import OSM
from shapely.wkt import loads as wkt_loads
from scipy.stats import linregress

PROCESS FLOW

1. Download census tract data from https://www2.census.gov/geo/tiger/TIGER2024/BG/; note id from https://www2.census.gov/geo/tiger/TIGER_RD18/STATE/, store in tl folder
2. Download OSM data for a given state here https://download.geofabrik.de/north-america/us.html, save as osm/{state}-latest.osm.pbf
3. Download ACS data from https://data.census.gov/table/ACSDT5Y2022.B02001; filter to "every block group in {county of target city}; save as acs_{city} folder inside acs_income or acs_race, rename nothing inside
4. Run the following code block to process data
5. Add new city to list of cities in graph code, run graph code

In [ ]:
cities_to_ids = {
    "newyork": 36,
    "losangeles": "06",
    "chicago": 17,
    "houston": 48,
    "phoenix": "04",
    "philadelphia": 42,
    "sanantonio": 48,
    "sandiego": "06",
    "dallas": 48,
    "jacksonville": 12,
    "austin": 48,
    "fortworth": 48, 
    "sanjose": "06",
    "columbus": 39,
    "charlotte": 37,
    "indianapolis": 18,
    "sanfrancisco": "06",
    "seattle": 53,
    "denver": "08",
    "oklahomacity": 40,
    "nashville": 47,
    "washington": 11,
    "elpaso": 48,
    "lasvegas": 32,
    "boston": 25,
    "detroit": 26,
    "portland": 41,
    "louisville": 21,
    "memphis": 47,
    "baltimore": 24,
    "milwaukee": 55,
    "albuquerque": 35,
    "tucson": "04",
    "fresno": "06",
    "sacramento": "06",
    "mesa": "04", 
    "atlanta": 13,
    "kansascity": 29
}

cities_to_states = {
    "newyork": "new-york",
    "losangeles": "california",
    "chicago": "illinois",
    "houston": "texas",
    "phoenix": "arizona",
    "philadelphia": "pennsylvania",
    "sanantonio": "texas",
    "sandiego": "california",
    "dallas": "texas",
    "jacksonville": "florida",
    "austin": "texas",
    "fortworth": "texas", 
    "sanjose": "california",
    "columbus": "ohio",
    "charlotte": "north-carolina",
    "indianapolis": "indiana",
    "sanfrancisco": "california",
    "seattle": "washington",
    "denver": "colorado",
    "oklahomacity": "oklahoma",
    "nashville": "tennessee",
    "washington": "district-of-columbia",
    "elpaso": "texas",
    "lasvegas": "nevada",
    "boston": "massachusetts",
    "detroit": "michigan",
    "portland": "oregon",
    "louisville": "kentucky",
    "memphis": "tennessee",
    "baltimore": "maryland",
    "milwaukee": "wisconsin",
    "albuquerque": "new-mexico",
    "tucson": "arizona",
    "fresno": "california",
    "sacramento": "california",
    "mesa": "arizona", 
    "atlanta": "georgia",
    "kansascity": "missouri"
}

In [ ]:
def convert_to_geojson(city, formal_city):
    
    # Access locally stored OSM files
    city_fp = f"osm/{cities_to_states[city]}-latest.osm.pbf"

    osm = OSM(city_fp)

    city_boundary = ox.geocode_to_gdf(f"{formal_city}, USA")
    print("Boundary defined")

    city_geom = city_boundary.geometry.iloc[0]

    tags = {
        "amenity": ["library", "fire_station", "bank", "place_of_worship", "pharmacy", "social_facility", "police", "community_centre"],
        "leisure": ["park"],
        "building": ["school", "hospital", "residential", "house", "apartments"],
        "landuse": ["residential"],
        "shop": ["supermarket"],
        "highway": ["bus_stop"]
    }

    gdf = osm.get_data_by_custom_criteria(custom_filter=tags, filter_type="keep")

    # Filter data to keep only those within the city boundary
    if gdf is not None and not gdf.empty:
        try:
            gdf = gdf[gdf.geometry.is_valid]
            gdf = gdf[gdf.intersects(city_geom)]
            gdf = gdf.drop(columns=["id", "timestamp"], errors="ignore")
            print("Data extracted successfully")
        except:
            invalid = gdf.loc[~gdf.geometry.is_valid]
            print(invalid)

        # Save to a GeoJSON file
        output_fp = f"buildings/{city}_buildings.geojson"
        gdf.to_file(output_fp, driver="GeoJSON")
        print(f"GeoJSON saved at {output_fp}")
    else:
        print("No data extracted for the specified tags.")


In [ ]:
%%time
# GENERALIZABLE FUNCTION

def process_data(city, id):

    census_tracts = gpd.read_file(f'tl/tl_2024_{id}_bg/tl_2024_{id}_bg.shp')
    
    # Load the OSM data (make sure it includes location data as points or polygons)
    osm_data = gpd.read_file(f'buildings/{city}_buildings.geojson')
        

    # Reproject if needed (make sure both are in the same CRS)
    osm_data = osm_data.to_crs(census_tracts.crs)
    
    # Perform spatial join
    osm_with_geoid = gpd.sjoin(osm_data, census_tracts[['GEOID', 'geometry']], how='left')
    
    # Load the ACS data
    acs_data = pd.read_csv(f"acs/acs_{city}/ACSDT5Y2022.B19013-Data.csv")
    
    acs_data = acs_data.rename(columns={"B02001_002E": "White"})
    acs_data = acs_data.rename(columns={"B02001_001E": "Total"})
    acs_data = acs_data.drop(0)
    acs_data["WhitePerc"] = acs_data["White"].astype(int) / acs_data["Total"].astype(int)
    acs_data = acs_data[['GEO_ID', 'WhitePerc']]
    acs_data["GEO_ID"] = acs_data["GEO_ID"].apply(lambda x: x[9:])
    acs_data["WhitePerc"] = acs_data["WhitePerc"].apply(lambda x: float('nan') if x == "-" else x)
    
    acs_race = acs_data.astype(str)
    acs_race = acs_race.dropna(subset=["WhitePerc"])
    
    # Now merge the data
    combined_with_race = osm_with_geoid.merge(acs_race, left_on="GEOID", right_on="GEO_ID", how="left")

    combined_with_race = combined_with_race.dropna(subset=["WhitePerc"])
        
    return combined_with_race

In [ ]:
%%time

cities = cities_to_ids.keys()

formal_cities = ["New York, New York", "Los Angeles, California", "Chicago, Illinois", "Houston, Texas", "Phoenix, Arizona", "Philadelphia, Pennsylvania", "San Antonio, Texas", "San Diego, California",
                 "Dallas, Texas", "Jacksonville, Florida", "Austin, Texas", "Fort Worth, Texas", "San Jose, California", "Columbus, Ohio", "Charlotte, North Carolina", "Indianapolis, Indiana", "San Francisco, California",
                 "Seattle, Washington", "Denver, Colorado", "Oklahoma City, Oklahoma", "Nashville, Tennessee", "Washington, District of Columbia", "El Paso, Texas", "Las Vegas, Nevada", "Boston, Massachusetts",
                 "Detroit, Michigan", "Portland, Oregon", "Louisville, Kentucky", "Memphis, Tennessee", "Baltimore, Maryland", "Milwaukee, Wisconsin", "Albuquerque, New Mexico", "Tucson, Arizona", "Fresno, California",
                 "Sacramento, California", "Mesa, Arizona", "Atlanta, Georgia", "Kansas City, Missouri"]

In [ ]:
dataframes = []

for city, formal_city in zip(cities_to_ids.keys(), formal_cities):
    print("starting", city, formal_city)
    if not os.path.exists(f"buildings/{city}_buildings.geojson"):
        convert_to_geojson(city, formal_city)
    dataframes[city] = process_data(city, cities_to_ids[city])
    print(city, "complete! length", len(dataframes[city]))

In [ ]:
'''
This num_divisions parameter is very important; it impacts the behavior of all future graphs. If you want graphs to appear as tertiles
(as in the lower correlative analysis), it should be 3; to produce proper graphs for distributions, it should be 5. This can be customized
to work with any reasonable number.
'''

def category_normalized_counts_by_race(categories, city_names, num_divisions=3):
    all_results = {}

    for city_name in city_names:
        
        print(city_name)
        
        df = dataframes[city_name]
        
        df.drop(['race_range'], axis=1, errors='ignore', inplace=True)
        
        df["WhitePerc"] = pd.to_numeric(df["WhitePerc"], errors="coerce")
        
        # Drop rows where WhitePerc is NaN after conversion
        df = df.dropna(subset=["WhitePerc"])

        # Step 1: Calculate race tertiles based on unique block groups
        unique_bg = df.drop_duplicates(subset=["GEO_ID"])[["GEO_ID", "WhitePerc"]]
    
        unique_bg["race_range"], race_bins = pd.qcut(
            unique_bg["WhitePerc"], 
            q=num_divisions, 
            retbins=True, 
            labels=[f"Q{i+1}" for i in range(num_divisions)]
        )
        
        # Step 2: Count the number of block groups in each race range
        block_group_counts = unique_bg.groupby("race_range", observed=False).size().reset_index(name="bg_count") 

        # Step 3: Merge race range back to the main DataFrame
        df = pd.merge(df, unique_bg[["GEO_ID", "race_range"]], on="GEO_ID", how="left")
        city_results = {}
        for category, items in categories.items():
            category_results = {}
            for item in items:
                try:
                    filtered_data = df[df[category] == item]

                    counts_by_race = filtered_data.groupby("race_range", observed=False).size().reset_index(name="count")

                    counts_with_bg = pd.merge(counts_by_race, block_group_counts, on="race_range", how="left")

                    # Calculate the normalized count (service count per block group) across cities
                    counts_with_bg["normalized_count"] = counts_with_bg["count"] / counts_with_bg["bg_count"]

                    # Store the results for each item in the current category
                    category_results[item] = counts_with_bg[["race_range", "normalized_count"]]
                except:
                    continue
            
            city_results[category] = category_results
        
        all_results[city_name] = city_results
        dataframes[city_name] = df
    
    return all_results, race_bins

# Take means of all trend lines to aggregate
def aggregate_city_trendlines(results, categories):
    aggregated_results = {}

    for category, items in categories.items():
        aggregated_results[category] = {}
        
        for item in items:
            all_city_data = []
            
            for city_name, city_data in results.items():
                try:
                    all_city_data.append(city_data[category][item])
                except:
                    continue
            
            combined_df = pd.concat(all_city_data)
            
            # Compute the average normalized count per race tertile
            aggregated_df = combined_df.groupby("race_range", observed=False)["normalized_count"].mean().reset_index()
            
            aggregated_results[category][item] = aggregated_df

    return aggregated_results

categories = {
    "amenity": ["library", "fire_station", "bank", "place_of_worship", "pharmacy", "social_facility", "police"],
    "leisure": ["park"],
    "building": ["school", "hospital"],
    "shop": ["supermarket"],
    "highway": ["bus_stop"]
}

city_names = list(cities_to_ids.keys())

results, race_bins = category_normalized_counts_by_race(categories, city_names)

aggregated_results = aggregate_city_trendlines(results, categories)

total_items = sum(len(items) for items in categories.values())
cols = 3  # Number of columns in the grid
rows = math.ceil(total_items / cols)  # Calculate the number of rows required

fig, axs = plt.subplots(rows, cols, figsize=(15, rows * 5))

plot_index = 0

# Loop through each category and item
for category, items in categories.items():
    for item in items:
        # Determine the current row and column based on plot_index
        row = plot_index // cols
        col = plot_index % cols
        ax = axs[row, col] if rows > 1 else axs[col]  

        # Plot aggregated data (single trendline per category)
        data = aggregated_results[category][item]
        ax.plot(data["race_range"], data["normalized_count"], marker='o', linestyle='-', label=f"{item} Trendline", color="black")

        ax.set_title(f"{category.capitalize()}: {item.capitalize()}")
        ax.set_xlabel("Percent White Tertile")
        ax.set_ylabel("Average Amenity Count : Block Group Ratio")
        ax.legend()
        
        plot_index += 1

# Hide any unused subplots if the grid has extra cells
for j in range(plot_index, rows * cols):
    fig.delaxes(axs[j // cols, j % cols])

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the main title
plt.show()


In [ ]:
classifications = {
    "quality-of-life": {
        "amenity": ["library", "place_of_worship"],
        "building": [],
        "leisure": ["park"],
        "shop": ["supermarket"],
        "highway": []
    },
    "economic-mobility": {
        "amenity": ["social_facility", "bank"],
        "building": ["school"],
        "leisure": [],
        "shop": [],
        "highway": ["bus_stop"]
    },
    "health-and-safety": {
        "amenity": ["fire_station", "police", "pharmacy"],
        "building": ["hospital"],
        "leisure": [],
        "shop": [],
        "highway": []
    }
}

In [ ]:
%%time

def calculate_min_distances_block_group(df, city_name, categories, block_group_column="GEO_ID"):
    projected_crs = "EPSG:3857"
    results = {}

    if df["geometry"].dtype == "object":
        df["geometry"] = df["geometry"].apply(
            lambda geom: wkt_loads(geom) if isinstance(geom, str) else geom
        )

    df = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    # Reproject to the projected CRS for distance calculations
    df = df.to_crs(projected_crs)
    
    city_id = cities_to_ids[city_name]
    census_tracts = gpd.read_file(f'tl/tl_2024_{city_id}_bg/tl_2024_{city_id}_bg.shp')
    census_tracts = census_tracts.to_crs(epsg = "4269")
    census_tracts = census_tracts[["GEOID", "geometry"]]

    block_group_areas = census_tracts.dissolve(by="GEOID")

    # Calculate area for each dissolved region
    
    block_group_areas = block_group_areas['geometry'].area.reset_index()

    block_group_areas.rename(columns={0: "block_group_area"}, inplace=True)
    
    block_group_areas.rename(columns={"GEOID": "GEO_ID"}, inplace=True)
    
    max_val = block_group_areas["block_group_area"].max()
    block_group_areas["norm_factor"] = 1 - (np.log1p(block_group_areas["block_group_area"]) / np.log1p(max_val))
    
    for category, items in categories.items():
        category_results = {}
        for item in items:
            # Filter amenities by category and item
            amenities = df[df[category] == item]
            amenities = gpd.GeoDataFrame(amenities, geometry="geometry").to_crs(projected_crs)
            amenities["geometry"] = amenities["geometry"].apply(
                lambda geom: geom.centroid if geom.geom_type == "Polygon" else geom
            )

            residential_buildings = df[
                (df["building"].isin(["residential", "house", "apartments"])) |  
                (df["landuse"] == "residential")  
            ].copy()

            # Randomly sample one residence per block group
            sampled_residences = residential_buildings.groupby(block_group_column).sample(n=1, random_state=42)

            # Compute minimum distance from sampled residences to each amenity
            sampled_residences[f"min_dist_to_{item}"] = sampled_residences["geometry"].apply(
                lambda geom: amenities.distance(geom).min() if not amenities.empty else np.nan
            )

            # Merge with block group area data
            sampled_residences = sampled_residences.merge(block_group_areas, on=block_group_column, how="left")

            # Normalize distances by block group size
            sampled_residences[f"normalized_dist_to_{item}"] = (
                sampled_residences[f"min_dist_to_{item}"] / np.sqrt(sampled_residences["block_group_area"])
            )
            
            scaling_factor = sampled_residences[f"min_dist_to_{item}"].mean() / sampled_residences[f"normalized_dist_to_{item}"].mean()
            sampled_residences[f"normalized_dist_to_{item}"] *= scaling_factor

            # Calculate average normalized distance per block group
            block_group_avg = sampled_residences.groupby(block_group_column, observed=False).agg(
                avg_dist=("normalized_dist_to_" + item, "mean"),
                race_range=("race_range", "first")
            ).reset_index()

            # Aggregate by race range (ignoring NaN values)
            race_range_avg = block_group_avg.groupby("race_range", observed=False).agg(
                avg_dist=("avg_dist", "mean")
            ).reset_index()

            category_results[item] = race_range_avg

        results[category] = category_results

    return results


def aggregate_distance_trendlines(results_by_city, categories):
    aggregated_results = {}

    for category, items in categories.items():
        aggregated_results[category] = {}

        for item in items:
            all_city_data = []

            for city_name, city_results in results_by_city.items():
                try:
                    city_data = city_results[category][item]
                    city_data = city_data.replace([np.inf, -np.inf], np.nan)  
                    all_city_data.append(city_data)
                except:
                    continue

            if all_city_data:
                combined_df = pd.concat(all_city_data, ignore_index=True)

                # Compute the average distance per race tertile (excluding NaNs)
                aggregated_df = combined_df.groupby("race_range", observed=False)["avg_dist"].mean().reset_index()

                aggregated_results[category][item] = aggregated_df

    return aggregated_results

def bootstrap_aggregate_distance_trendlines(results_by_city, categories, n_iterations=500, sample_size=None):
    aggregated_results = {}
    all_city_names = list(results_by_city.keys())
    if sample_size is None:
        sample_size = len(all_city_names)
        
    for category, items in categories.items():
        aggregated_results[category] = {}
        
        for item in items:
            bootstrap_results = []
            
            for i in range(n_iterations):
                sampled_cities = np.random.choice(all_city_names, size=sample_size, replace=True)
                all_city_data = []
                
                for city in sampled_cities:
                    city_results = results_by_city.get(city, None)
                    if city_results is None:
                        continue
                    try:
                        city_data = city_results[category][item]
                        city_data = city_data.replace([np.inf, -np.inf], np.nan)
                        all_city_data.append(city_data)
                    except Exception as e:
                        continue
                
                if all_city_data:
                    combined_df = pd.concat(all_city_data, ignore_index=True)
                    group_means = combined_df.groupby("race_range", observed=False)["avg_dist"].mean().reset_index()
                    bootstrap_results.append(group_means)
            
            all_race_ranges = sorted(set().union(*(set(df['race_range']) for df in bootstrap_results)))
            summary_data = []
            
            for race_range in all_race_ranges:
                values = []
                for df in bootstrap_results:
                    val = df.loc[df["race_range"] == race_range, "avg_dist"]
                    if not val.empty:
                        values.append(val.iloc[0])
                if values:
                    mean_val = np.mean(values)
                    lower = np.percentile(values, 2.5)
                    upper = np.percentile(values, 97.5)
                    summary_data.append({
                        "race_range": race_range,
                        "avg_dist": mean_val,
                        "ci_lower": lower,
                        "ci_upper": upper
                    })
            
            summary_df = pd.DataFrame(summary_data)
            aggregated_results[category][item] = summary_df
            
    return aggregated_results

def prepare_distance_data(results_dict, categories):
    results = {}
    for category, items in categories.items():
        category_results = {}
        for item in items:
            if item in results_dict[category]:
                category_results[item] = results_dict[category][item]
        results[category] = category_results
    return results

def plot_aggregated_distance_graphs(aggregated_results, categories):
    total_items = sum(len(items) for items in categories.values())
    cols = 3
    rows = math.ceil(total_items / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(15, rows * 5))
    plot_index = 0
    
    rows_cols_by_amenity = {
        "pharmacy": (0, 2, "Health & Safety"),
        "police": (1, 2, "Health & Safety"),
        "hospital": (2, 2, "Health & Safety"),
        "fire_station": (3, 2, "Health & Safety"),
        "school": (0, 1, "Economic Mobility"),
        "bank": (1, 1, "Economic Mobility"),
        "social_facility": (2, 1, "Economic Mobility"),
        "bus_stop": (3, 1, "Economic Mobility"),
        "park": (0, 0, "Quality of Life"),
        "supermarket": (1, 0, "Quality of Life"),
        "place_of_worship": (2, 0, "Quality of Life"),
        "library": (3, 0, "Quality of Life")
    }

    for category, items in categories.items():
        for item in items:
            row, col, cat = rows_cols_by_amenity.get(item, (0,0))
            ax = axs[row, col] if rows > 1 else axs[col]
            
            if item == "social_facility":
                name = "Social Facility"
            elif item == "place_of_worship":
                name = "Place of Worship"
            elif item == "fire_station":
                name = "Fire Station"
            elif item == "bus_stop":
                name = "Bus Stop"
            else:
                name = item.capitalize()

            data = aggregated_results.get(category, {}).get(item, None)
            if data is not None and not data.empty:
                
                if col == 0:
                    color = "green"
                elif col == 1:
                    color = "brown"
                else:
                    color = "blue"
                
                x = data["race_range"]
                y = data["avg_dist"].values
                
                ax.plot(x, y, marker='o', linestyle='-', color=color, label=f"{item} Trendline")
                
                ax.fill_between(
                    x, 
                    data["ci_lower"], 
                    data["ci_upper"], 
                    color=color, 
                    alpha=0.05  
                )
                
                x_numeric = np.arange(len(data))
                slope, intercept, r_value, p_value, std_err = linregress(x_numeric, y)
                print(f"📊 {category.capitalize()} - {item}:")
                print(f"  Slope: {slope:.3f}")
                print(f"  Intercept: {intercept:.3f}")
                print(f"  R-squared: {r_value**2:.3f}")
                print(f"  P-value: {p_value:.3e}")
                print(f"  Std Err: {std_err:.3f}\n")
            
            ax.set_title(f"{cat}: {name}")
            ax.set_xlabel("Percent White")
            ax.set_ylabel("Average Adjusted Distance (Meters)")
            plot_index += 1

    # Remove any empty subplots
    for j in range(plot_index, rows * cols):
        fig.delaxes(axs[j // cols, j % cols])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# List of city dataframes
city_names = list(cities_to_ids.keys())

with open("results_by_city_5.pkl", "rb") as f:
    results_by_city = pickle.load(f)

# Aggregate data for trendlines (ignoring missing amenities)
with open("aggregated_results_race_5.pkl", "rb") as f:
    aggregated_results = pickle.load(f)

plot_aggregated_distance_graphs(aggregated_results, categories)


In [ ]:
def plot_classification_average_trendlines(aggregated_results, classifications):

    classification_names = list(classifications.keys())
    print(classification_names)
    n_class = len(classification_names)
    
    fig, axs = plt.subplots(1, n_class, figsize=(6 * n_class, 5), sharey=True)
    if n_class == 1:
        axs = [axs] 
    
    flat_results = {}
    for outer_key in aggregated_results:
        for item, df in aggregated_results[outer_key].items():
            flat_results[item] = df

    for ax, cls in zip(axs, classification_names):
        items = []
        for group_type, item_list in classifications[cls].items():
            items.extend(item_list)
        
        dfs = [flat_results[item] for item in items if item in flat_results]
        
        if not dfs:
            ax.set_title(cls.replace("-", " ").capitalize() + "\n(No Data)")
            continue
        
        combined_df = pd.concat(dfs, ignore_index=True)
        
        summary_df = combined_df.groupby("race_range", as_index=False)[["avg_dist", "ci_lower", "ci_upper"]].mean()
        summary_df = summary_df.sort_values("race_range")
        
        if cls == "quality-of-life":
            color = "green"
            name = "Quality of Life"
        elif cls == "economic-mobility":
            color = "brown"
            name = "Economic Mobility"
        else:
            color = "blue"
            name = "Health and Safety"
        
        x = summary_df["race_range"]
        y = summary_df["avg_dist"]
        ax.plot(x, y, marker='o', linestyle='-', color=color, label=f"{cls} trendline")
        
        ax.set_title(name)
        ax.set_xlabel("Percent White")
        ax.set_ylabel("Average Adjusted Distance (Meters)")
    
    plt.tight_layout()
    plt.show()


In [ ]:
plot_classification_average_trendlines(aggregated_results, classifications)

In [ ]:
def aggregate_by_gini_index(results_by_city, categories, cities_to_gini):

    aggregated_results = {}

    for category, items in categories.items():
        aggregated_results[category] = {}

        for item in items:
            city_data = []  

            for city_name, city_results in results_by_city.items():
                if city_name not in cities_to_gini:
                    continue  

                gini = cities_to_gini[city_name]  
                
                try:
                    city_df = city_results[category][item]  
                    city_df = city_df.replace([np.inf, -np.inf], np.nan).dropna()

                    avg_distance = city_df["avg_dist"].mean()  # Compute mean distance
                    city_data.append({"gini_index": gini, "avg_dist": avg_distance})
                except:
                    continue  

            if city_data:
                df = pd.DataFrame(city_data)

                lower_ci = np.percentile(df["avg_dist"], 2.5) if len(df) > 1 else df["avg_dist"].min()
                upper_ci = np.percentile(df["avg_dist"], 97.5) if len(df) > 1 else df["avg_dist"].max()
                
                df["ci_lower"] = lower_ci
                df["ci_upper"] = upper_ci

                aggregated_results[category][item] = df

    return aggregated_results


In [ ]:
def plot_gini_vs_distance(aggregated_results, categories):
    total_items = sum(len(items) for items in categories.values())
    cols = 3
    rows = math.ceil(total_items / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(15, rows * 5))
    plot_index = 0
    
    rows_cols_by_amenity = {
        "pharmacy": (0, 2, "Health & Safety"),
        "police": (1, 2, "Health & Safety"),
        "hospital": (2, 2, "Health & Safety"),
        "fire_station": (3, 2, "Health & Safety"),
        "school": (0, 1, "Economic Mobility"),
        "bank": (1, 1, "Economic Mobility"),
        "social_facility": (2, 1, "Economic Mobility"),
        "bus_stop": (3, 1, "Economic Mobility"),
        "park": (0, 0, "Quality of Life"),
        "supermarket": (1, 0, "Quality of Life"),
        "place_of_worship": (2, 0, "Quality of Life"),
        "library": (3, 0, "Quality of Life")
    }

    for category, items in categories.items():
        for item in items:
            row, col, cat = rows_cols_by_amenity.get(item, (0, 0, "Other"))
            ax = axs[row, col] if rows > 1 else axs[col]

            # Get the aggregated data
            data = aggregated_results.get(category, {}).get(item, None)
            if data is not None and not data.empty:
                # Sort data by Gini index
                data = data.sort_values("gini_index")

                # Linear regression analysis
                slope, intercept, r_value, p_value, std_err = linregress(data["gini_index"], data["avg_dist"])

                # Generate regression line
                x_vals = np.linspace(data["gini_index"].min(), data["gini_index"].max(), 100)
                y_vals = slope * x_vals + intercept

                # Assign color based on category
                color = "green" if col == 0 else "brown" if col == 1 else "blue"

                # Plot the regression line
                ax.plot(x_vals, y_vals, linestyle="-", color=color, label=f"{item} Trendline")

                # Display regression statistics
                ax.annotate(
                    f"Slope: {slope:.3f}\nR²: {r_value**2:.3f}\nP-value: {p_value:.3e}",
                    xy=(0.05, 0.85), xycoords="axes fraction", fontsize=10,
                    bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.3")
                )

                print(f"📊 {category.capitalize()} - {item}:")
                print(f"  Slope: {slope:.3f}")
                print(f"  R-squared: {r_value**2:.3f}")
                print(f"  P-value: {p_value:.3e}")
                print(f"  Std Err: {std_err:.3f}\n")

            ax.set_title(f"{cat}: {item.capitalize()}")
            ax.set_xlabel("Gini Index")
            ax.set_ylabel("Average Adjusted Distance (Meters)")
            plot_index += 1

    for j in range(plot_index, rows * cols):
        fig.delaxes(axs[j // cols, j % cols])

    plt.tight_layout()
    plt.show()



In [ ]:
aggregated_results_gini = aggregate_by_gini_index(results_by_city, categories, cities_to_gini)
plot_gini_vs_distance(aggregated_results_gini, categories)

In [ ]:
def aggregate_category_by_gini(results_by_city, classifications, cities_to_gini):

    aggregated_results = {}

    for category, amenities in classifications.items():
        category_data = []

        for city_name, city_results in results_by_city.items():
            if city_name not in cities_to_gini:
                continue  

            gini = cities_to_gini[city_name]  
            city_avg_distances = []

            for amenity_group, items in amenities.items():
                for item in items:
                    try:
                        city_df = city_results[amenity_group][item]
                        city_df = city_df.replace([np.inf, -np.inf], np.nan).dropna()
                        avg_distance = city_df["avg_dist"].mean()  
                        city_avg_distances.append(avg_distance)
                    except:
                        continue  

            if city_avg_distances:
                category_avg_dist = np.mean(city_avg_distances)  
                category_data.append({"gini_index": gini, "avg_dist": category_avg_dist})

        if category_data:
            df = pd.DataFrame(category_data)

            lower_ci = np.percentile(df["avg_dist"], 2.5) if len(df) > 1 else df["avg_dist"].min()
            upper_ci = np.percentile(df["avg_dist"], 97.5) if len(df) > 1 else df["avg_dist"].max()
            
            df["ci_lower"] = lower_ci
            df["ci_upper"] = upper_ci

            aggregated_results[category] = df

    return aggregated_results

aggregated_results = aggregate_category_by_gini(results_by_city, classifications, cities_to_gini)

In [ ]:
def plot_category_trendlines(aggregated_results):

    classification_names = list(aggregated_results.keys())
    n_class = len(classification_names)

    fig, axs = plt.subplots(1, n_class, figsize=(6 * n_class, 5), sharey=True)
    if n_class == 1:
        axs = [axs]  

    for ax, cls in zip(axs, classification_names):
        data = aggregated_results.get(cls, None)
        if data is None or data.empty:
            ax.set_title(cls.replace("-", " ").capitalize() + "\n(No Data)")
            continue

        data = data.sort_values("gini_index")

        slope, intercept, r_value, p_value, std_err = linregress(data["gini_index"], data["avg_dist"])

        x_vals = np.linspace(data["gini_index"].min(), data["gini_index"].max(), 100)
        y_vals = slope * x_vals + intercept

        color_map = {
            "quality-of-life": "green",
            "economic-mobility": "brown",
            "health-safety": "blue"
        }
        color = color_map.get(cls, "blue")

        ax.plot(x_vals, y_vals, linestyle="-", color=color, label=f"{cls} Trendline")

        ax.annotate(
            f"Slope: {slope:.3f}\nR²: {r_value**2:.3f}\nP-value: {p_value:.3e}",
            xy=(0.05, 0.85), xycoords="axes fraction", fontsize=10,
            bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.3")
        )

        print(f"📊 {cls.capitalize()} Category:")
        print(f"  Slope: {slope:.3f}")
        print(f"  R-squared: {r_value**2:.3f}")
        print(f"  P-value: {p_value:.3e}")
        print(f"  Std Err: {std_err:.3f}\n")

        ax.set_title(cls.replace("-", " ").capitalize())
        ax.set_xlabel("Gini Index")
        ax.set_ylabel("Average Adjusted Distance (Meters)")

    plt.tight_layout()
    plt.show()

plot_category_trendlines(aggregated_results)

In [ ]:
# from Census table B19083
cities_to_gini = {
    "newyork": 0.5136,
    "losangeles": 0.4928,
    "chicago": 0.4817,
    "houston": 0.4827,
    "phoenix": 0.4534,
    "philadelphia": 0.4828,
    "sanantonio": 0.4600,
    "sandiego": 0.4602,
    "dallas": 0.4637,
    "jacksonville": 0.4663,
    "austin": 0.4563,
    "fortworth": 0.4637,
    "sanjose": 0.4743,
    "columbus": 0.4583,
    "charlotte": 0.4749,
    "indianapolis": 0.4655,
    "sanfrancisco": 0.4913,
    "seattle": 0.4620,
    "denver": 0.4482,
    "oklahomacity": 0.4646,
    "nashville": 0.4646,
    "washington": 0.4472,
    "elpaso": 0.4663,
    "lasvegas": 0.4706,
    "boston": 0.4847,
    "detroit": 0.4726,
    "portland": 0.4491,
    "louisville": 0.4624,
    "memphis": 0.4897,
    "baltimore": 0.4603,
    "milwaukee": 0.4710,
    "albuquerque": 0.4668,
    "tucson": 0.4679,
    "fresno": 0.4730,
    "sacramento": 0.4556,
    "mesa": 0.4564,
    "atlanta": 0.4676,
    "kansascity": 0.4518,
}

In [ ]:
city_names = cities_to_ids.keys()

class_counts = {}

for city in city_names:
    class_counts[city] = {
        "CCU": {
            "quality-of-life": 0,
            "economic-mobility": 0,
            "health-and-safety": 0,
        },
        "CCD": {
            "quality-of-life": 0,
            "economic-mobility": 0,
            "health-and-safety": 0,
        },
        "LP": {
            "quality-of-life": 0,
            "economic-mobility": 0,
            "health-and-safety": 0,
        },
        "LN": {
            "quality-of-life": 0,
            "economic-mobility": 0,
            "health-and-safety": 0,
        }
    }
    
    for category, items in categories.items():
        for item in items:
            try:
                distances = list(results_by_city[city][category][item]["avg_dist"])
            except:
                continue
            q1 = distances[0]
            q2 = distances[1]
            q3 = distances[2]
            
            if item in classifications["quality-of-life"][category]:
                purpose = "quality-of-life"
            elif item in classifications["economic-mobility"][category]:
                purpose = "economic-mobility"
            else:
                purpose = "health-and-safety"
            
            if q1 > q2 < q3:
                class_counts[city]["CCU"][purpose] += 1
            elif q1 < q2 > q3:
                class_counts[city]["CCD"][purpose] += 1
            elif q1 < q2 < q3:
                class_counts[city]["LP"][purpose] += 1
            elif q1 > q2 > q3:
                class_counts[city]["LN"][purpose] += 1

In [ ]:
mux = pd.MultiIndex.from_product([city_names, ["CCU", "CCD", "LP", "LN"], ["quality-of-life", "economic-mobility", "health-and-safety"]])
df = pd.DataFrame(class_counts, columns=mux)

In [ ]:
flattened_data = []

for city, categories in class_counts.items():
    for category, metrics in categories.items():
        for metric, value in metrics.items():
            flattened_data.append((city, category, metric, value))

df = pd.DataFrame(flattened_data, columns=["City", "Category", "Metric", "Value"])

df_pivot = df.pivot(index="City", columns=["Metric", "Category"], values="Value")

In [ ]:
category_counts = {"CCD": {}, "CCU": {}, "LP": {}, "LN": {}}

for city, categories in class_counts.items():
    
    city_totals = {}
    for category, metrics in categories.items():
        category_counts[category][city] = sum(metrics.values())

counts_df = pd.DataFrame(category_counts)

In [ ]:
df_avg = df.groupby(["Metric", "Category"])["Value"].mean().reset_index()

metrics = df_avg["Metric"].unique()

fig, axs = plt.subplots(1, len(metrics), figsize=(15, 6), sharey=True)

for i, metric in enumerate(metrics):
    metric_data = df_avg[df_avg["Metric"] == metric]
    axs[i].bar(metric_data["Category"], metric_data["Value"], color='skyblue')
    axs[i].set_title(f"{metric.capitalize()}")
    axs[i].set_xlabel("Category")
    axs[i].set_ylabel("Average # of trendlines (out of 4)")

plt.tight_layout()
plt.show()

In [ ]:
cities_to_founding = {
    "newyork": 1624,
    "losangeles": 1781,
    "chicago": 1803,
    "houston": 1837,
    "phoenix": 1868,
    "philadelphia": 1681,
    "sanantonio": 1718,
    "sandiego": 1769,
    "dallas": 1841,
    "jacksonville": 1791,
    "austin": 1835,
    "fortworth": 1849, 
    "sanjose": 1777,
    "columbus": 1812,
    "charlotte": 1755,
    "indianapolis": 1821,
    "sanfrancisco": 1776,
    "seattle": 1851,
    "denver": 1858,
    "oklahomacity": 1889,
    "nashville": 1779,
    "washington": 1790,
    "elpaso": 1659,
    "lasvegas": 1911,
    "boston": 1630,
    "detroit": 1701,
    "portland": 1845,
    "louisville": 1778,
    "memphis": 1819,
    "baltimore": 1729,
    "milwaukee": 1833,
    "albuquerque": 1706,
    "tucson": 1775,
    "fresno": 1872,
    "sacramento": 1848,
    "mesa": 1878, 
    "atlanta": 1843,
    "kansascity": 1838
}

In [ ]:
df_pivot["Founding Date"] = df_pivot.index.get_level_values("City").map(cities_to_founding)

df_sorted = df_pivot.reset_index().sort_values(by="Founding Date").set_index(["City"])

df_sorted.head()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

amenities = ["quality-of-life", "economic-mobility", "health-and-safety"]

trendlines = ["CCU", "CCD", "LN", "LP"]

colors = {"CCU": "blue", "CCD": "green", "LN": "orange", "LP": "purple"}

for i, amenity in enumerate(amenities):
    ax = axs[i]
    
    test_df = df_sorted.loc[:, df_sorted.columns.get_level_values('Metric').isin([amenity, 'Founding Date'])]
    test_df = test_df.loc[:, test_df.columns.get_level_values("Category").isin(trendlines + [''])]

    for category in trendlines:
        founding_dates = test_df["Founding Date"].values
        values = test_df[(amenity, category)].values.flatten()

        slope, intercept, r_value, p_value, std_err = linregress(founding_dates, values)

        trendline = slope * founding_dates + intercept
        
        alpha_threshold = 0.05
        
        line_alpha = 1.0 if p_value < alpha_threshold else 0.3
        
        ax.plot(founding_dates, trendline, color=colors[category], linestyle='--', label=f"{category} Trendline", alpha=line_alpha)
        
        # Print statistical results
        print(f"{amenity} - {category}:")
        print(f"  Slope: {slope:.3f}")
        print(f"  Intercept: {intercept:.3f}")
        print(f"  R-squared: {r_value**2:.3f}")
        print(f"  P-value: {p_value:.3e}")
        print(f"  Std Err: {std_err:.3f}")
    
    # Customize the subplot
    ax.set_title(f"{amenity.capitalize()}")
    ax.set_xlabel("Founding Date")
    if i == 0:
        ax.set_ylabel("# of Trendlines (Out of 4)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
cities_to_partisanship = {
    "newyork": 100,
    "losangeles": 90,
    "chicago": 90,
    "houston": 0,
    "phoenix": 20,
    "philadelphia": 70,
    "sanantonio": 0,
    "sandiego": 90,
    "dallas": 0,
    "jacksonville": 30,
    "austin": 0,
    "fortworth": 0,
    "sanjose": 90,
    "columbus": 40,
    "charlotte": 10,
    "indianapolis": 10,
    "sanfrancisco": 90,
    "seattle": 10,
    "denver": 60,
    "oklahomacity": 0,
    "nashville": 20,
    "washington": 10,
    "elpaso": 0,
    "lasvegas": 60,
    "boston": 10,
    "detroit": 70,
    "portland": 10,
    "louisville": 20,
    "memphis": 20,
    "baltimore": 90,
    "milwaukee": 80,
    "albuquerque": 80,
    "tucson": 20,
    "fresno": 90,
    "sacramento": 90,
    "mesa": 20,
    "atlanta": 20,
    "kansascity": 2
}

In [ ]:
df_pivot["Partisanship"] = df_pivot.index.get_level_values("City").map(cities_to_partisanship)

df_sorted = df_pivot.reset_index().sort_values(by="Partisanship").set_index(["City"])

df_sorted.head()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

amenities = ["quality-of-life", "economic-mobility", "health-and-safety"]

trendlines = ["CCU", "CCD", "LN", "LP"]

colors = {"CCU": "blue", "CCD": "green", "LN": "orange", "LP": "purple"}

for i, amenity in enumerate(amenities):
    ax = axs[i]
    
    test_df = df_sorted.loc[:, df_sorted.columns.get_level_values('Metric').isin([amenity, 'Partisanship'])]
    test_df = test_df.loc[:, test_df.columns.get_level_values("Category").isin(trendlines + [''])]

    for category in trendlines:
        populations = test_df["Partisanship"].values
        values = test_df[(amenity, category)].values.flatten()

        slope, intercept, r_value, p_value, std_err = linregress(populations, values)

        trendline = slope * populations + intercept
        
        alpha_threshold = 0.05
        
        line_alpha = 1.0 if p_value < alpha_threshold else 0.3
        
        ax.plot(populations, trendline, color=colors[category], linestyle='--', label=f"{category} Trendline", alpha=line_alpha)
        
        print(f"{amenity} - {category}:")
        print(f"  Slope: {slope:.3f}")
        print(f"  Intercept: {intercept:.3f}")
        print(f"  R-squared: {r_value**2:.3f}")
        print(f"  P-value: {p_value:.3e}")
        print(f"  Std Err: {std_err:.3f}")
    
    ax.set_title(f"{amenity.capitalize()}")
    ax.set_xlabel("Percent Presidential Elections with Democrat Majority (Last 10 Elections)")
    if i == 0:
        ax.set_ylabel("# of Trendlines (Out of 4)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
cities_to_distance = {
    "newyork": 40.6943,
    "losangeles": 34.1141,
    "chicago": 41.8375,
    "houston": 29.786,
    "phoenix": 33.5722,
    "philadelphia": 40.0077,
    "sanantonio": 29.4632,
    "sandiego": 33.0094,
    "dallas": 32.7935,
    "jacksonville": 30.3322,
    "austin": 30.3005,
    "fortworth": 32.7817,
    "sanjose": 37.3012,
    "columbus": 39.9862,
    "charlotte": 35.2083,
    "indianapolis": 39.7771,
    "sanfrancisco": 37.7558,
    "seattle": 47.6211,
    "denver": 39.762,
    "oklahomacity": 35.4676,
    "nashville": 36.1715,
    "washington": 38.9047,
    "elpaso": 31.8476,
    "lasvegas": 36.2333,
    "boston": 42.3188,
    "detroit": 42.3834,
    "portland": 45.5371,
    "louisville": 38.1663,
    "memphis": 35.1087,
    "baltimore": 39.3051,
    "milwaukee": 43.0642,
    "albuquerque": 35.1054,
    "tucson": 32.1541,
    "fresno": 36.783,
    "sacramento": 38.5677,
    "mesa": 33.4015,
    "atlanta": 33.7628,
    "kansascity": 39.1238,
}

In [ ]:
df_pivot["Distance"] = df_pivot.index.get_level_values("City").map(cities_to_distance)

df_sorted = df_pivot.reset_index().sort_values(by="Distance").set_index(["City"])

df_sorted.head()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

amenities = ["quality-of-life", "economic-mobility", "health-and-safety"]

colors = {"CCU": "blue", "CCD": "green", "LN": "orange", "LP": "purple"}

for i, amenity in enumerate(amenities):
    ax = axs[i]
    
    test_df = df_sorted.loc[:, df_sorted.columns.get_level_values('Metric').isin([amenity, 'Distance'])]
    test_df = test_df.loc[:, test_df.columns.get_level_values("Category").isin(trendlines + [''])]
    
    for category in trendlines:
        founding_dates = test_df["Distance"].values
        values = test_df[(amenity, category)].values.flatten()

        slope, intercept, r_value, p_value, std_err = linregress(founding_dates, values)
        
        trendline = slope * founding_dates + intercept
        
        alpha_threshold = 0.05
        
        line_alpha = 1.0 if p_value < alpha_threshold else 0.3
        
        ax.plot(founding_dates, trendline, color=colors[category], linestyle='--', label=f"{category} Trendline", alpha=line_alpha)
        
        print(f"{amenity} - {category}:")
        print(f"  Slope: {slope:.3f}")
        print(f"  Intercept: {intercept:.3f}")
        print(f"  R-squared: {r_value**2:.3f}")
        print(f"  P-value: {p_value:.3e}")
        print(f"  Std Err: {std_err:.3f}")
    
    ax.set_title(f"{amenity.capitalize()}")
    ax.set_xlabel("Latitude")
    if i == 0:
        ax.set_ylabel("# of Trendlines (Out of 4)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
cities_to_gini = {
    "newyork": 0.5136,
    "losangeles": 0.4928,
    "chicago": 0.4817,
    "houston": 0.4827,
    "phoenix": 0.4534,
    "philadelphia": 0.4828,
    "sanantonio": 0.4600,
    "sandiego": 0.4602,
    "dallas": 0.4637,
    "jacksonville": 0.4663,
    "austin": 0.4563,
    "fortworth": 0.4637,
    "sanjose": 0.4743,
    "columbus": 0.4583,
    "charlotte": 0.4749,
    "indianapolis": 0.4655,
    "sanfrancisco": 0.4913,
    "seattle": 0.4620,
    "denver": 0.4482,
    "oklahomacity": 0.4646,
    "nashville": 0.4646,
    "washington": 0.4472,
    "elpaso": 0.4663,
    "lasvegas": 0.4706,
    "boston": 0.4847,
    "detroit": 0.4726,
    "portland": 0.4491,
    "louisville": 0.4624,
    "memphis": 0.4897,
    "baltimore": 0.4603,
    "milwaukee": 0.4710,
    "albuquerque": 0.4668,
    "tucson": 0.4679,
    "fresno": 0.4730,
    "sacramento": 0.4556,
    "mesa": 0.4564,
    "atlanta": 0.4676,
    "kansascity": 0.4518,
}

In [ ]:
df_pivot["Gini"] = df_pivot.index.get_level_values("City").map(cities_to_gini)

df_sorted = df_pivot.reset_index().sort_values(by="Gini").set_index(["City"])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

amenities = ["quality-of-life", "economic-mobility", "health-and-safety"]

colors = {"CCU": "blue", "CCD": "green", "LN": "orange", "LP": "purple"}

for i, amenity in enumerate(amenities):
    ax = axs[i]
    
    test_df = df_sorted.loc[:, df_sorted.columns.get_level_values('Metric').isin([amenity, 'Gini'])]
    test_df = test_df.loc[:, test_df.columns.get_level_values("Category").isin(trendlines + [''])]
    
    for category in trendlines:
        founding_dates = test_df["Gini"].values
        values = test_df[(amenity, category)].values.flatten()

        slope, intercept, r_value, p_value, std_err = linregress(founding_dates, values)
        
        trendline = slope * founding_dates + intercept
        
        alpha_threshold = 0.05
        
        line_alpha = 1.0 if p_value < alpha_threshold else 0.3
        
        ax.plot(founding_dates, trendline, color=colors[category], linestyle='--', label=f"{category} Trendline", alpha=line_alpha)
        
        print(f"{amenity} - {category}:")
        print(f"  Slope: {slope:.3f}")
        print(f"  Intercept: {intercept:.3f}")
        print(f"  R-squared: {r_value**2:.3f}")
        print(f"  P-value: {p_value:.3e}")
        print(f"  Std Err: {std_err:.3f}")
    
    ax.set_title(f"{amenity.capitalize()}")
    ax.set_xlabel("Gini Index")
    if i == 0:
        ax.set_ylabel("# of Trendlines (Out of 4)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()